In [1]:
import csv

# CSVファイルを開く
with open('/Users/shuma/Lecture/programing/dsprogramming-muro/nittere_news.csv', mode='r', encoding='utf-8') as file:
    csv_reader = csv.reader(file)

    # ヘッダー行をスキップする場合
    next(csv_reader)

    # 行ごとに読み込む
    for row in csv_reader:
        print(row)


['1月2日', 'JALの炎上事故救助活動Live中継']
['1月3日', '海保機の侵入の許可なしと国交省が通信記録を発表']
['1月4日', '菅制官の「海保機の侵入に気づかなかった」という説明']
['1月5日', 'カメラが捉えた衝突直前の海保機、一部始終']
['1月6日', '避難生活の取材']
['1月7日', '岸田首相が池田議員の逮捕を受けコメント']
